# 1 
Loading the Data

In [23]:
import pandas as pd

df_raw = pd.read_csv("raw_euribor_3month_data.csv", header=None)
print(df_raw)

             0                 1
0         Date  Euribor 3 months
1   21/03/2025             2.386
2   20/03/2025             2.387
3   19/03/2025             2.411
4   18/03/2025             2.426
..         ...               ...
61  27/12/2024             2.683
62  26/12/2024               NaN
63  25/12/2024               NaN
64  24/12/2024               NaN
65  23/12/2024             2.731

[66 rows x 2 columns]


# 2 
We interpolate between missing values - I was not able to get data for all dates

In [24]:
import pandas as pd

# 1) Rename the columns for clarity
df_raw.columns = ["Date", "Euribor_3m"]

# 2) Parse the 'Date' column as datetime. 
#    If your dates are in day/month/year format, set dayfirst=True.
df_raw["Date"] = pd.to_datetime(df_raw["Date"], dayfirst=True, errors="coerce")

# 3) Convert the Euribor values to numeric (floats), coercing non-numeric to NaN
df_raw["Euribor_3m"] = pd.to_numeric(df_raw["Euribor_3m"], errors="coerce")

# 4) Drop any rows where 'Date' could not be parsed
df_raw.dropna(subset=["Date"], inplace=True)

# 5) Set 'Date' as the index and sort by it, so time-based interpolation can work
df_raw.set_index("Date", inplace=True)
df_raw.sort_index(inplace=True)

# 6) Interpolate missing values in 'Euribor_3m' based on time
#    This will fill NaN by looking at neighboring valid dates.
df_raw["Euribor_3m"] = df_raw["Euribor_3m"].interpolate(method="time")

# 7) (Optional) Check the result
print(df_raw.head(20))
print("\nNumber of missing values after interpolation:")
print(df_raw["Euribor_3m"].isna().sum())

            Euribor_3m
Date                  
2024-12-23    2.731000
2024-12-24    2.719000
2024-12-25    2.707000
2024-12-26    2.695000
2024-12-27    2.683000
2024-12-30    2.706250
2024-12-31    2.714000
2025-01-01    2.719667
2025-01-02    2.725333
2025-01-03    2.731000
2025-01-06    2.839000
2025-01-07    2.875000
2025-01-08    2.832000
2025-01-09    2.789000
2025-01-10    2.788000
2025-01-13    2.785000
2025-01-14    2.766500
2025-01-15    2.748000
2025-01-16    2.726000
2025-01-17    2.704000

Number of missing values after interpolation:
0


C:\Users\Alex\AppData\Local\Temp\ipykernel_19204\1270932479.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw["Date"] = pd.to_datetime(df_raw["Date"], dayfirst=True, errors="coerce")


In [25]:
df_raw.to_csv("cleaned_euribor_3month_data.csv")